PI de europa: bdsice (de momento descarga de ficheros. Hay que automatizar)

PI de EEUU fred
PI de UK https://www.ons.gov.uk/economy/economicoutputandproductivity/output/timeseries/k22a/diop

bono 10 años USA diario https://fred.stlouisfed.org/series/DGS10

BAMLH0A0HYM2ICE BofA US High Yield Index Option-Adjusted Spread (BAMLH0A0HYM2)

en FRED GDP
Argentina NGDPRSAXDCARQ desde 2004
Brasil NGDPRSAXDCBRQ desde 1996
Mexico NGDPRSAXDCMXQ desde 1993
Chile en https://www.bcentral.cl/es/areas/estadisticas/cuentas-nacionales-trimestrales (buscar otro, pero en principio la serie está desde 1997)
Colombia en https://www.dane.gov.co/index.php/estadisticas-por-tema/cuentas-nacionales/cuentas-nacionales-trimestrales/pib-informacion-tecnica pero desde 2005
Perú https://estadisticas.bcrp.gob.pe/estadisticas/series/trimestrales/resultados/PN02526AQ/html, en tasas interanuales por trimestres desde 1980




Importación PInd

In [44]:
import pandas as pd
import numpy as np
import re

# Tu código de carga y limpieza se mantiene igual...
pi_eu = pd.read_csv('../data/indicadores/IPI_UE.txt', sep =";", encoding='latin1')
pi_eu = pi_eu.drop(pi_eu.index[0])
pi_eu = pi_eu.set_index(pi_eu.columns[0])
pi_eu.index = pd.to_datetime(pi_eu.index, format='%Y%m')
pi_eu.index.name = 'fecha'
pi_eu = pi_eu.loc[:, ~pi_eu.columns.str.startswith('Unnamed')]
pi_eu = pi_eu.apply(pd.to_numeric, errors='coerce')

pi_usa = pd.read_csv('../data/indicadores/PI_USA_fred.csv', sep =",", encoding='latin1')
pi_usa = pi_usa.set_index(pi_usa.columns[0])
pi_usa.index = pd.to_datetime(pi_usa.index).strftime('%Y%m')  # String YYYYMM

pi_uk = pd.read_excel('../data/indicadores/PI_UK_SO.xlsx')
patron = r'^\d{4} [A-Za-z]{3}$'
pi_uk = pi_uk[pi_uk['Title'].str.match(patron, na=False)]
pi_uk = pi_uk.set_index('Title')
pi_uk.index = pi_uk.index.str.title()

def fecha_a_yyyymm(fecha_str):
    meses = {
        'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
        'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
        'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
    }
    partes = fecha_str.split()
    if len(partes) == 2:
        año = partes[0]
        mes = partes[1]
        if mes in meses:
            return año + meses[mes]
    return None

pi_uk.index = [fecha_a_yyyymm(idx) for idx in pi_uk.index]
pi_uk.index.name = 'fecha'

# Después de renombrar las columnas de pi_uk
pi_uk = pi_uk.rename(columns={pi_uk.columns[0]: 'PI_UK'})

# Convertir la columna PI_UK a float64
pi_uk['PI_UK'] = pd.to_numeric(pi_uk['PI_UK'], errors='coerce').astype('float64')

# ESTANDARIZAR TODOS LOS ÍNDICES A STRING YYYYMM ANTES DEL MERGE
pi_eu.index = pi_eu.index.strftime('%Y%m')  # Convertir datetime a string YYYYMM
# pi_usa ya está en formato string YYYYMM
# pi_uk ya está en formato string YYYYMM

# Renombrar las columnas
pi_usa = pi_usa.rename(columns={pi_usa.columns[0]: 'PI_USA'})
pi_eu = pi_eu.rename(columns={
    pi_eu.columns[0]: 'PI_FRA',
    pi_eu.columns[1]: 'PI_GER', 
    pi_eu.columns[2]: 'PI_ITA'
})
pi_uk = pi_uk.rename(columns={pi_uk.columns[0]: 'PI_UK'})

# Unir todos los dataframes usando el índice (fechas)
df_combined = pi_usa.join([pi_eu, pi_uk], how='outer')

# Filtrar datos desde enero de 2000 usando string
df_combined = df_combined[df_combined.index >= '200001']

# Ordenar por fecha
df_combined = df_combined.sort_index()

# Renombrar el índice
df_combined.index.name = 'fecha'

print(f"DataFrame combinado: {df_combined.shape}")
print(f"Período: {df_combined.index.min()} - {df_combined.index.max()}")
print(f"Columnas: {df_combined.columns.tolist()}")

DataFrame combinado: (304, 5)
Período: 200001 - 202504
Columnas: ['PI_USA', 'PI_FRA', 'PI_GER', 'PI_ITA', 'PI_UK']


Bono 10 años


In [45]:

dgs10 = pd.read_csv('../data/indicadores/DGS10.csv', sep =",", encoding='latin1')
dgs10 = dgs10.set_index(dgs10.columns[0])
dgs10.index = pd.to_datetime(dgs10.index).strftime('%Y%m')  # String YYYYMM

# Calcular la media mensual ignorando valores NaN
dgs10_monthly = dgs10.groupby(dgs10.index).mean()

# Renombrar la columna si es necesario
dgs10_monthly = dgs10_monthly.rename(columns={dgs10_monthly.columns[0]: 'DGS10'})

df_combined = df_combined.join(dgs10_monthly, how='left')



BofA US High Yield Index Option-Adjusted Spread (BAMLH0A0HYM2)

In [46]:
spreadusa = pd.read_csv('../data/indicadores/BAMLH0A0HYM2.csv', sep =",", encoding='latin1')
spreadusa = spreadusa.set_index(spreadusa.columns[0])
spreadusa.index = pd.to_datetime(spreadusa.index).strftime('%Y%m')  # String YYYYMM

spreadusa = spreadusa.rename(columns={spreadusa.columns[0]: 'SPREAD_USA'})
spreadusa['SPREAD_USA'] = spreadusa['SPREAD_USA'].interpolate(method='linear')

spreadusa_monthly = spreadusa.groupby(spreadusa.index).mean()

df_combined = df_combined.join(spreadusa_monthly, how='left')

In [48]:
df_combined.to_csv('../data/indicadores/indicadores_macroeconomicos.csv', sep=';', encoding='latin1')